# Bayesian Camera Calibration
> Let's apply Bayesian analysis to calibrate a camera

- toc: true 
- badges: true
- comments: true
- categories: [Bayesian, Computer Vision]
- image: images/2020-03-28-Bayesian-Camera-Calibration/header.jpg

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pymc3 as pm

plt.rcParams['figure.figsize'] = [10,10]

In [2]:
def x_rot(theta,x,y,z):
    theta *= np.pi/180
    x_rot = x
    y_rot = np.cos(theta)*y - np.sin(theta)*z
    z_rot = np.sin(theta)*y + np.cos(theta)*z
    return(x_rot,y_rot,z_rot)

def y_rot(theta,x,y,z):
    theta *= np.pi/180
    x_rot = np.cos(theta)*x + np.sin(theta)*z
    y_rot = y
    z_rot = -np.sin(theta)*x + np.cos(theta)*z
    return(x_rot,y_rot,z_rot)

def z_rot(theta,x,y,z):
    theta *= np.pi/180
    x_rot = np.cos(theta)*x - np.sin(theta)*y
    y_rot = np.sin(theta)*x + np.cos(theta)*y
    z_rot = z
    return(x_rot,y_rot,z_rot)


points = np.loadtxt("data/2020-02-23-An-Adventure-In-Camera-Calibration/points.csv")

points_2d = points[:,0:2]
points_3d = points[:,2:5]

number_points = points.shape[0]

px = points_2d[:,0]
py = points_2d[:,1]

X_input = points_3d[:,0]
Y_input = points_3d[:,1]
Z_input = points_3d[:,2]

In [3]:
def rotate(theta_Z_est,theta_Y_est,theta_X_est, X_est, Y_est, Z_est):
    X_est, Y_est, Z_est = z_rot(theta_Z_est, X_est, Y_est, Z_est)
    X_est, Y_est, Z_est = y_rot(theta_Y_est, X_est, Y_est, Z_est)
    X_est, Y_est, Z_est = x_rot(theta_X_est, X_est, Y_est, Z_est)
    return(X_est, Y_est, Z_est)


    # Define priors
    X_translate_est = pm.Normal('X_translate', mu = -7, sigma = 1)
    Y_translate_est = pm.Normal('Y_translate', mu = -13, sigma = 1)
    Z_translate_est = pm.Normal('Z_translate', mu = 3, sigma = 1)
    focal_length_est = pm.Normal('focal_length',mu = 1000, sigma = 100)
    
    theta_Z_est = pm.Normal('theta_Z',mu = -45, sigma = 30)
    theta_Y_est = pm.Normal('theta_Y',mu = 0, sigma = 15)
    theta_X_est = pm.Normal('theta_X',mu = 90, sigma = 30)
    
    c_x_est = pm.Normal('c_x',mu = 1038.42, sigma = 100)
    c_y_est = pm.Normal('c_y',mu = 2666.56, sigma = 100)
    
    k1 = -0.351113
    k2 = 0.185768
    k3 = -0.032289
    
    error_scale = 2
    
    X_est = X_input + X_translate_est
    Y_est = Y_input + Y_translate_est
    Z_est = Z_input + Z_translate_est
    
    X_est, Y_est, Z_est = rotate(theta_Z_est, theta_Y_est, theta_X_est, X_est, Y_est, Z_est)
    
    px_est = X_est / Z_est
    py_est = Y_est / Z_est
    
    r = np.sqrt(px_est**2 + py_est**2)
    px_est *= (1 + k1 * r + k2 * r**2 + k3 * r**3)
    py_est *= (1 + k1 * r + k2 * r**2 + k3 * r**3)
    
    px_est *= focal_length_est
    py_est *= focal_length_est

    px_est += c_x_est
    py_est += c_y_est

    delta = np.sqrt((px - px_est)**2 + (py - py_est)**2)
    
    # Define likelihood
    likelihood = pm.Normal('error', mu = delta, sigma = error_scale, observed=np.zeros(number_points))

    # Inference!
    trace = pm.sample(2_000, cores=4, tune=5000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [c_y, c_x, theta_X, theta_Y, theta_Z, focal_length, Z_translate, Y_translate, X_translate]
Sampling 4 chains, 0 divergences:   1%|          | 269/28000 [00:01<02:16, 202.49draws/s] 


RuntimeError: Chain 2 failed.

In [ ]:
plt.figure(figsize=(7, 7))
pm.traceplot(trace[1000:])
plt.tight_layout();

In [ ]:
pm.plot_posterior(trace);

In [ ]:
pm.summary(trace)